In [1]:
import astroprov

from astropy.io import fits
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.table import Table
from astropy.table import Column
from astropy.io import fits
from astropy.io.ascii import SExtractor
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy import wcs

from astroquery.simbad import Simbad

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from photutils import DAOStarFinder
from photutils import CircularAperture
import os
import subprocess

import image_registration
from image_registration import chi2_shift
from image_registration.fft_tools import shift

In [17]:
imageDirectory = '/data/mj1e16/kepler/cal/'
dirlist = os.listdir(imageDirectory)
dirlist = [x for x in dirlist if x[0] != '.']
#dirlist = sort(dirlist)
imagelist = [imageDirectory+x for x in dirlist]
#print(dirlist)

In [4]:
def correctExtensionOrder(imageName):
    hdu_list = fits.open(imageName)
    extensionOrder = np.zeros(len(hdu_list))
    for channel in range(1,len(hdu_list)):
        hdr = hdu_list[channel].header
        location = hdr['SKYGROUP'] # independant location
        extensionOrder[location] = channel # ordered from 0-84 channel with extension in place of name
    return extensionOrder

In [5]:
def medianImages(image1,image2,channelOrder1,channelOrder2,outfile): # medianing images 2 at a time as they are large
    image_concat = []
    medianlist = []
    hdu_list_1 = fits.open(image1)
    hdu_list_2 = fits.open(image2)
    for channel in range(1,len(channelOrder1)):
        chanNo1 = int(channelOrder1[channel])
        chanNo2 = int(channelOrder2[channel])
        
        im1 = hdu_list_1[chanNo1].data
        im2 = hdu_list_2[chanNo2].data
        
        yoff,xoff = image_registration.cross_correlation_shifts(im1,im2)
        im1_aligned_to_im2 = np.roll(np.roll(im1,int(yoff),1),int(xoff),0)

        medianImage = np.median([im2,im1_aligned_to_im2],axis=0)
        hdu = fits.PrimaryHDU(medianImage)
        hdu.writeto(outfile+'_{}.fits'.format(channel), clobber=True)
        medianlist.append(outfile+'_{}.fits'.format(channel))
    return outfile

In [6]:
def reMedian(median,image2,channel,channelOrder2,outfile): # medianing images 2 at a time as they are large
    
    image_concat = []
    hdu_list_1 = fits.open(median)
    hdu_list_2 = fits.open(image2)
    chanNo2 = int(channelOrder2[channel])
    im1 = hdu_list_1[0].data
    im2 = hdu_list_2[chanNo2].data
    
    yoff,xoff = image_registration.cross_correlation_shifts(im1,im2)
    im1_aligned_to_im2 = np.roll(np.roll(im1,int(yoff),1),int(xoff),0)
    
    medianImage = np.median([im2,im1_aligned_to_im2],axis=0)
    hdu = fits.PrimaryHDU(medianImage)
    hdu.writeto(outfile, clobber=True)
    
    return outfile

In [7]:
extensionOrders = []
for image in imagelist:
    extensionOrders.append(correctExtensionOrder(image))

In [26]:
medianDir = '/data/mj1e16/kepler/newDiff/'
medians = os.listdir(medianDir)
medians = [x for x in medians if x[0] != '.']
medians.sort()

In [67]:
medianImages(imagelist[0],imagelist[1],extensionOrders[0],extensionOrders[1],'/data/mj1e16/kepler/newDiff/median')

'/data/mj1e16/kepler/newDiff/median'

In [68]:
for imageNo in range(2,len(imagelist)):
    for channel in range(1,len(medians)):
        medianFile = medianDir+'median_{}.fits'.format(channel)
        reMedian(medianFile,imagelist[imageNo],channel,extensionOrders[imageNo],medianFile)

In [ ]:
os.chdir('/data/mj1e16/kepler/hotpants-master/')
for imNo in range(len(imagelist)):
    for channel in range(1,len(medians)):
        ccdChannel = int(extensionOrders[imNo][channel])
        inim = imagelist[imNo]+'[{}]'.format(ccdChannel)
        tmplim = medianDir+'median_{}.fits'.format(channel)
        outim = '/data/mj1e16/kepler/properDiff/diff_imNo{}_channel{}.fits'.format(imNo,channel)
        subprocess.call(['./hotpants','-inim',inim,'-tmplim',tmplim,'-outim',outim,'-v','0'])